In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# imports
import pandas as pd
import itertools
import os
from tqdm import tqdm
import time

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()
from auctions.webutils import *
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from open_spiel.python.examples.ubc_dispatch import verify_config, get_cluster_details, write_and_submit, dispatch_eval_database, dispatch_single_br_database

plt.style.use('https://raw.githubusercontent.com/gregdeon/plots/main/style.mplstyle')
plt.rcParams['axes.titlesize'] = 7
plt.rcParams['figure.dpi'] = 300

In [3]:
# Step 1: Find a game that you want to do this on. 
# (Ensure you have a the rho=0 game)

experiment = 'feb8_3p_5supply_v2'
run_name = 'feb8_3p_5supply_feb8_3p_5supply_3_base_dev1000_rho1_t3-cfr_10external_plus_linear-100'
run_name_tb = 'feb8_3p_5supply_feb8_3p_5supply_3_base_dev1000_rho1_t3_tie_break-cfr_10external_plus_linear-100'

run = EquilibriumSolverRun.objects.get(name=run_name, experiment__name=experiment)
run_tb = EquilibriumSolverRun.objects.get(name=run_name_tb, experiment__name=experiment)

In [4]:
Experiment.objects.get(name='feb8_3p_5supply_v2').equilibriumsolverrun_set.all()

<QuerySet [<EquilibriumSolverRun: feb8_3p_5supply_feb8_3p_5supply_3_base_dev1000_rho0_t3_tie_break-cfr_10_no_tremexternal_plus_linear_no_trem-104 (feb8_3p_5supply_v2)>, <EquilibriumSolverRun: feb8_3p_5supply_feb8_3p_5supply_2_base_dev1000_rho0_t3_tie_break-cfr_10_no_tremexternal_plus_linear_no_trem-101 (feb8_3p_5supply_v2)>, <EquilibriumSolverRun: feb8_3p_5supply_feb8_3p_5supply_0_base_dev1000_rho1_t3-cfr_10external_plus_linear-100 (feb8_3p_5supply_v2)>, <EquilibriumSolverRun: feb8_3p_5supply_feb8_3p_5supply_2_base_dev1000_rho0_t3_tie_break-cfr_10_no_tremexternal_plus_linear_no_trem-104 (feb8_3p_5supply_v2)>, <EquilibriumSolverRun: feb8_3p_5supply_feb8_3p_5supply_0_base_dev1000_rho1_t3-cfr_10external_plus_linear-102 (feb8_3p_5supply_v2)>, <EquilibriumSolverRun: feb8_3p_5supply_feb8_3p_5supply_3_base_dev1000_rho0_t3_tie_break-cfr_10_no_tremexternal_plus_linear_no_trem-100 (feb8_3p_5supply_v2)>, <EquilibriumSolverRun: feb8_3p_5supply_feb8_3p_5supply_0_base_dev1000_rho1_t3-cfr_10external_

In [ ]:
print(run.equilibriumsolverruncheckpoint_set.last().get_modal_eval().nash_conv)
print(run.equilibriumsolverruncheckpoint_set.last().get_modal_eval().heuristic_conv)
print(run_tb.equilibriumsolverruncheckpoint_set.last().get_modal_eval().nash_conv)
print(run_tb.equilibriumsolverruncheckpoint_set.last().get_modal_eval().heuristic_conv)

In [8]:
# Step 2: Code that will run HC and NC for longer 
BASE_OUTPUT_DIR = os.environ['CLOCK_AUCTION_OUTPUT_ROOT']
BR_DIR = 'best_responses'

def dispatch_one_off_command(cmd, experiment_name, run_name, extra_name='', cluster='ada_cpu_short', submit=True):
    spiel_path, config_dir, pydir, manage_path = verify_config()
    cluster_details = get_cluster_details(cluster)
    command = f'python {manage_path} {cmd}'

    slurm_job_name = f'one_off_{experiment_name}_{run_name}_{extra_name}'
    job_file_text = f"""{cluster_details['shell']}
{cluster_details['preamble']}
#SBATCH --job-name={slurm_job_name}
#SBATCH --time=0-3:00:00 # days-hh:mm:ss
#SBATCH -e slurm-%j-{slurm_job_name}.err
#SBATCH -o slurm-%j-{slurm_job_name}.out

{cluster_details['load_py']}
export OPENSPIEL_PATH={spiel_path}

CMD=`{command}`
echo $CMD
eval $CMD
"""

    experiment_dir = f'{BASE_OUTPUT_DIR}/{experiment_name}/{run_name}/{BR_DIR}'
    write_and_submit(experiment_dir, slurm_job_name, job_file_text, submit)

    logging.info(f"Dispatched experiment!")

# for r in [run, run_tb]:
#     for heuristic in [True, False]:
#         cp = r.equilibriumsolverruncheckpoint_set.last()
#         cmd = f"nashconv --time_limit_seconds 3600 --t {cp.t} --run_name {cp.equilibrium_solver_run.name} --experiment_name {cp.equilibrium_solver_run.experiment.name} --heuristic {heuristic}"
#         dispatch_one_off_command(cmd, cp.equilibrium_solver_run.experiment.name, cp.equilibrium_solver_run.name, extra_name='heuristic' if heuristic else 'nc')

In [9]:
#### ALL RUNS ###
for r in Experiment.objects.get(name='feb12_3p_ppo_v1').equilibriumsolverrun_set.all():
# for r in Experiment.objects.get(name='feb8_3p_5supply_v2').equilibriumsolverrun_set.all():
    # for heuristic in [True, False]:
    for heuristic in [False]:
        cp = r.equilibriumsolverruncheckpoint_set.last()
        cmd = f"nashconv --time_limit_seconds 3600 --t {cp.t} --run_name {cp.equilibrium_solver_run.name} --experiment_name {cp.equilibrium_solver_run.experiment.name} --heuristic {heuristic}"
        dispatch_one_off_command(cmd, cp.equilibrium_solver_run.experiment.name, cp.equilibrium_solver_run.name, extra_name='heuristic' if heuristic else 'nc')



Submitted batch job 1091677
Submitted batch job 1091678
Submitted batch job 1091679
Submitted batch job 1091680
Submitted batch job 1091681
Submitted batch job 1091682
Submitted batch job 1091683
Submitted batch job 1091684
Submitted batch job 1091685
Submitted batch job 1091686
Submitted batch job 1091687
Submitted batch job 1091688
Submitted batch job 1091689
Submitted batch job 1091690
Submitted batch job 1091691
Submitted batch job 1091692
Submitted batch job 1091693
Submitted batch job 1091694
Submitted batch job 1091695
Submitted batch job 1091696
Submitted batch job 1091697
Submitted batch job 1091698
Submitted batch job 1091699
Submitted batch job 1091700
Submitted batch job 1091701
Submitted batch job 1091702
Submitted batch job 1091703
Submitted batch job 1091704
Submitted batch job 1091705
Submitted batch job 1091706
Submitted batch job 1091707
Submitted batch job 1091708
Submitted batch job 1091709
Submitted batch job 1091710
Submitted batch job 1091711
Submitted batch job 

In [ ]:
# Step 4: Code that will run PPO-BR 
config = 'ppo_jun8_23/ppo_76'

for r in [run, run_tb]:
    game = r.game.load_as_spiel()
    for br_player in range(game.num_players()):
        cp = r.equilibriumsolverruncheckpoint_set.last()
        experiment_name = cp.equilibrium_solver_run.experiment.name
        run_name = cp.equilibrium_solver_run.name
        t = cp.t
        dispatch_single_br_database(experiment_name, run_name, t, br_player, config, True, '--total_timesteps 500', django_command='ppo_br', cluster='ada_cpu_short')

In [ ]:
# Step 5: Code that will run single agent MCCFR
# Require a game wrapper

In [ ]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "one_off"  | wc -l

In [ ]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "one_off" | awk '{print $1}' | xargs scancel

In [ ]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "ppo_br" 

In [ ]:
!sinfo

In [6]:
from django.db import connection
with connection.cursor() as c:
    c.execute("\dt+")
    c.fetchall()


ProgrammingError: syntax error at or near "\"
LINE 1: \dt+
        ^
